## Part #1: Parsing and preprocessing data

By using requests library, I get html code of the wikipedia page

Then, by using BeautifulSoup, table with top grossing films is being found, parsed and preprocessed

In [6]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find('table', {'class': 'wikitable'})

rows = table.find_all('tr')[1:]  # Skip header
films = []

BASE_WIKI_URL = "https://en.wikipedia.org"

def parse_film_page(link):
    response = requests.get(BASE_WIKI_URL + link)
    soup = BeautifulSoup(response.text, 'html.parser')

    infobox = soup.find('table', {'class': 'infobox'})
    director = "-"
    director_row = infobox.find('th', string='Directed by')
    if director_row:
        director = director_row.find_next('td').get_text(separator=', ', strip=True)

    country = "-"
    country_row = infobox.find('th', string=['Country', 'Countries'])
    if country_row:
        country = country_row.find_next('td').get_text(separator=', ', strip=True).split(', ')[0]

    return director, country

for row in rows:
    cols = row.find_all('td')
    
    title = row.find('i').find('a')['title'] 
    title = str(title).replace('[', '').replace(']', '')
    year = cols[3].text.strip()

    link_tag = row.find('i').find('a') if row.find('i') else None
    if link_tag:
        film_link = link_tag['href']
        director, country = parse_film_page(film_link)
    else:
        director = cols[2].text.strip().replace('\n', ', ')
        country = cols[4].text.strip()

    revenue = cols[2].text.strip().split('$')[-1]
    
    revenue = revenue.replace('$', '').replace(',', '')
    
    films.append({
        'title': title,
        'release_year': year,
        'director': director,
        'box_office': revenue,
        'country': country
    })

df = pd.DataFrame(films)

## Part #2: Saving data into the database

I chose sqlite for this purpose since there is no need to download anything for its use

In [7]:
conn = sqlite3.connect('../data/films.db')
cursor = conn.cursor()

# Create table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS films (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        release_year INT,
        director TEXT,
        box_office REAL,
        country TEXT
    )
''')

# Insert data
df.to_sql('films', conn, if_exists='replace', index=False)
conn.commit()
conn.close()

## Part #3: Saving sata into json file

Let's also save it into json

In [8]:
df.to_json('../data/films.json', orient='records', indent=2)